In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

In [2]:
import spotipy.util as util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Get songs from user input

In [3]:
cid = input("Give me cid")
secret = input("Give me secret")

Give me cidf9bbcbf7ac9e4f77b8f17ca9c5ed267e
Give me secret511452ccbd6a447ebfdb9a4b3fcd19f0


In [5]:
cid="f9bbcbf7ac9e4f77b8f17ca9c5ed267e"
secret = "511452ccbd6a447ebfdb9a4b3fcd19f0"

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [7]:
# search for a song
song_name = "Levitating"
artist_name = "Dua Lipa"
song_to_search = f"artist:{artist_name} track:{song_name}"
searchResults = sp.search(song_to_search, limit=1, offset=0, type="track")

In [8]:
song_id = searchResults['tracks']['items'][0]['id']

In [9]:
def get_song_features(track_ids):
    res = []
    for t in track_ids:
        res.append(sp.audio_features(tracks=t)[0])
    return res

In [10]:
df = get_song_features([song_id])

In [11]:
df

[{'danceability': 0.702,
  'energy': 0.825,
  'key': 6,
  'loudness': -3.787,
  'mode': 0,
  'speechiness': 0.0601,
  'acousticness': 0.00883,
  'instrumentalness': 0,
  'liveness': 0.0674,
  'valence': 0.915,
  'tempo': 102.977,
  'type': 'audio_features',
  'id': '5nujrmhLynf4yMoMtj8AQF',
  'uri': 'spotify:track:5nujrmhLynf4yMoMtj8AQF',
  'track_href': 'https://api.spotify.com/v1/tracks/5nujrmhLynf4yMoMtj8AQF',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5nujrmhLynf4yMoMtj8AQF',
  'duration_ms': 203064,
  'time_signature': 4}]

## Test finding 5 songs (scratched)

In [107]:
# def search_for_songs(song_names):
#     track_ids = []
#     for s in song_names:
#         searchResults = sp.search(s, 1, 0, "track")
#         song_id = searchResults['tracks']['items'][0]['id']
#         track_ids.append(song_id)
#     return get_song_features(track_ids)

In [108]:
# songs = ["Levitating","Blinding Lights","Ocean","Physical","Thank you next"]
# df_new = search_for_songs(songs)

# Find songs from the anime music dataset

In [21]:
path_to_anime = "../../data/external/anime_songs.csv"
anime_music = pd.read_csv(path_to_anime)

In [23]:
anime_music.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'name'],
      dtype='object')

In [90]:
useful_cols = ['danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness',
       'valence', 'tempo']
df_anime = anime_music[useful_cols]

In [13]:
df = pd.DataFrame.from_records(df,index=[0]*len(df))

In [118]:
df_new_song = df[useful_cols]

In [119]:
neigh = NearestNeighbors(n_neighbors=3, algorithm = 'auto').fit(df_anime)

In [120]:
idxs = neigh.kneighbors(df_new_song, 5, return_distance=False)

In [121]:
anime_music.iloc[list(idxs[0]),:]

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name
154,0,0.511,0.806,7,-3.433,1,0.0690,0.433000,0.000000,0.0965,0.552,102.070,audio_features,79AU26qJdGVLPK31RsWy31,spotify:track:79AU26qJdGVLPK31RsWy31,https://api.spotify.com/v1/tracks/79AU26qJdGVL...,https://api.spotify.com/v1/audio-analysis/79AU...,91240,4,salvation - Anime ver.
162,0,0.517,0.882,2,-5.507,1,0.0770,0.000079,0.000000,0.3360,0.342,103.003,audio_features,2FTt8Qkc04fcRZzwDEsNrh,spotify:track:2FTt8Qkc04fcRZzwDEsNrh,https://api.spotify.com/v1/tracks/2FTt8Qkc04fc...,https://api.spotify.com/v1/audio-analysis/2FTt...,250118,4,Get up! Shout!
77,0,0.592,0.769,2,-5.268,1,0.0311,0.319000,0.000005,0.1370,0.385,102.033,audio_features,4uVvqoI2sWncxvqgwRZ1H9,spotify:track:4uVvqoI2sWncxvqgwRZ1H9,https://api.spotify.com/v1/tracks/4uVvqoI2sWnc...,https://api.spotify.com/v1/audio-analysis/4uVv...,322147,4,ヒカレイノチ
25,0,0.669,0.751,5,-5.447,0,0.0299,0.004830,0.023700,0.3260,0.833,103.976,audio_features,1YvLvyEHEUv6xuablK3kpj,spotify:track:1YvLvyEHEUv6xuablK3kpj,https://api.spotify.com/v1/tracks/1YvLvyEHEUv6...,https://api.spotify.com/v1/audio-analysis/1YvL...,223846,4,空と虚
19,0,0.599,0.932,11,-5.622,1,0.0407,0.002150,0.000000,0.1460,0.539,102.065,audio_features,04LCCxG3pbOYeENiyUc7Xb,spotify:track:04LCCxG3pbOYeENiyUc7Xb,https://api.spotify.com/v1/tracks/04LCCxG3pbOY...,https://api.spotify.com/v1/audio-analysis/04LC...,208300,4,Black Rover


In [113]:
df_new_song

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,tempo
0,0.702,0.825,-3.787,0.0601,0.008830,0.000000,0.915,102.977
0,0.514,0.730,-5.934,0.0598,0.001460,0.000095,0.334,171.005
0,0.486,0.904,-3.751,0.0638,0.000034,0.000028,0.786,173.734
0,0.660,0.875,-3.333,0.0472,0.028100,0.001580,0.688,146.946
0,0.717,0.653,-5.634,0.0658,0.229000,0.000000,0.412,106.966


# Calculate distances between two dataframes

In [18]:
centroids = pd.read_csv("../../data/intermediate/centroids.csv")
df_song = df
features = ['danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness',
        'liveness','valence', 'tempo']

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(df_song[features],centroids[features])
dist

array([[ 0.20669405, -0.21392465,  0.01350239,  0.05708245,  0.12277632]])

In [38]:
int(centroids.loc[np.argmin(dist,axis=-1),"clusterId"])

1

In [29]:
df_anime = pd.read_csv("../../data/intermediate/clustered_songs.csv")

In [54]:
df_cluster = df_anime.loc[df_anime.clusterId==1,features]
dist = cosine_similarity(df_song[features],df_cluster)
inds = np.argpartition(dist,10,axis=-1)

df_cluster.reset_index(drop=True).loc[inds[0][:10]]


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
8,0.325,0.05830,-25.045,0.0440,0.986,0.9010,0.0968,0.0389,65.514
15,0.441,0.00742,-31.918,0.0393,0.993,0.9520,0.1030,0.1420,68.066
104,0.342,0.02870,-31.008,0.0411,0.986,0.7750,0.1790,0.1210,64.366
16,0.292,0.05660,-21.355,0.0355,0.994,0.8710,0.1010,0.0386,35.660
107,0.265,0.02820,-27.958,0.0476,0.985,0.8740,0.0941,0.1280,65.088
96,0.240,0.00772,-28.248,0.0452,0.974,0.9300,0.0756,0.0364,75.027
79,0.306,0.00653,-29.199,0.0394,0.958,0.9320,0.1280,0.2100,77.002
98,0.383,0.00467,-29.878,0.0528,0.991,0.8650,0.1090,0.2250,69.003
85,0.194,0.01740,-26.644,0.0378,0.983,0.0391,0.0845,0.0350,70.983
109,0.309,0.04230,-24.400,0.0565,0.993,0.9230,0.0893,0.2070,67.534
